# Import Statements

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns

# Code

> avg_columns: $(df, str) \rightarrow df$

**Parameters**: A dataframe object and a string for the suffix of the column.

**Returns**: A dataframe with the average for the columns.

> concat_lookup: $(df, str, str) \rightarrow Maybe \ df$

**Parameters**: A dataframe object, a string for the identifier of the lookup table and a string for the suffix of the lookup table.

**Returns**: A dataframe object with the result of merging the df parameter and the lookup that has the identifier received. Can return None if an exception is thrown during read_csv().

In [3]:
def avg_columns(df, suffix):
    avg_series = df.mean(axis = 1)
    return pd.DataFrame(avg_series, columns = ["avgValue"+suffix])

def concat_lookup(df, df_id, suffix = ''):  
  try:  
    new_df = (pd.read_csv('../datasets/lookup_'+df_id+suffix+'.csv', index_col = 0)
              .pipe(avg_columns, suffix = df_id)
    )
  except Exception as e:
    print(f'Error: unexpected error when reading file": {e}')
    return None
  return df.merge(new_df, how='left', left_index=True, right_index=True).fillna(0)


In [4]:
lookup_df = (pd.read_csv('../datasets/lookup_war.csv', index_col = 0)
    .pipe(concat_lookup, df_id = '1')
    .pipe(concat_lookup, df_id = '2')
    .pipe(concat_lookup, df_id = '3')
)
lookup_df

,degree,avgValue1,avgValue2,avgValue3
0x0000000000000000000000000000000000000000,579.0,2251.333333,1893.0,1288.333333
0x0000000000000000000000000000000000000001,16.0,5.000000,5.0,76.000000
0x0000000000000000000000000000000000000064,1.0,0.000000,0.0,0.333333
0x0000000000000000000000000000000000000800,2.0,0.000000,0.0,0.000000
0x0000000000000000000000000000000000001010,7.0,1.000000,5.0,3.666667
...,...,...,...,...
0xffffffff2ba8f66d4e51811c5190992176930278,133.0,2455.666667,871.0,492.333333
0xfffffffff15abf397da76f1dcc1a1604f45126db,67.0,1921.333333,407.5,191.666667
0xffffffffff2419497bf75e415bc9a7d446e05c0f,2.0,17.333333,55.0,13.000000
0xffffffffff402b1b62421a978cf93a56453d1496,1.0,0.000000,0.0,0.000000


> calc_df_distances: $(df, str) \rightarrow Maybe \ df$

**Parameters**: A dataframe object and a string containing the name of the target column.

**Returns**: A new dataframe with the target column and the distance between that column and all the others, or None if the target column does not exist.

In [7]:
def calc_df_distances(df_: pd.DataFrame, target_col: str):
  scaler = MinMaxScaler()
  df = df_.copy() #want to make the function immutable
  col_list = list(df.columns)
  if target_col not in col_list: return None
  scaled_data = scaler.fit_transform(df)
  df = pd.DataFrame(scaled_data, columns=col_list, index = df.index)
  col_list.remove(target_col)
  for col in col_list:
    target_val = df[target_col].values
    col_val = df[col].values
    new_col = np.abs(target_val - col_val)
    df[col+'_dist'] = new_col
    df.drop([col], axis = 1, inplace = True)
  df['total_dist'] = df.loc[:, df.columns != target_col].sum(axis = 1)
  return df

In [8]:
scaler = MinMaxScaler()
pd.DataFrame(scaler.fit_transform(lookup_df), columns = lookup_df.columns, index = lookup_df.index)['degree'].values

array([3.26877041e-04, 8.48296818e-06, 0.00000000e+00, ...,
       5.65531212e-07, 0.00000000e+00, 0.00000000e+00])

In [1]:
df = (
    lookup_df
    .pipe(calc_df_distances, target_col = 'degree')
)
df

NameError: name 'lookup_df' is not defined

Now we have $df$ ready to evaluate as we want.

In [10]:
dist_lower_bound = .7
df_filter = df.copy().query('total_dist >= 2.5 and total_dist < 3')
df_filter = df_filter.query('avgValue1_dist > '+str(dist_lower_bound))
df_filter = df_filter.query('avgValue2_dist > '+str(dist_lower_bound))
df_filter = df_filter.query('avgValue3_dist > '+str(dist_lower_bound))
df_filter.sort_values(by = 'total_dist', ascending = False).iloc[:10]

,degree,avgValue1_dist,avgValue2_dist,avgValue3_dist,total_dist
0x7f268357a8c2552623316e2562d90e642bb538e5,0.891518,0.891518,0.891518,0.891518,2.674554


Something else we can do is compare the distribution of degrees and degree changes between all periods and accounts.

### Comparing the degree distributions

In [5]:
test_df = lookup_df.copy()
test_df['avgValue1'] = test_df['avgValue1']/test_df['degree']
test_df['avgValue2'] = test_df['avgValue2']/test_df['degree']
test_df['avgValue3'] = test_df['avgValue3']/test_df['degree']
test_df['totalVal'] = test_df['avgValue1'] + test_df['avgValue2'] + test_df['avgValue3']
test_df['stdVal'] = test_df[['avgValue1', 'avgValue2', 'avgValue3']].std(axis = 1)
test_df

,degree,avgValue1,avgValue2,avgValue3,totalVal,stdVal
0x0000000000000000000000000000000000000000,579.0,3.888313,3.269430,2.225101,9.382844,0.840626
0x0000000000000000000000000000000000000001,16.0,0.312500,0.312500,4.750000,5.375000,2.561992
0x0000000000000000000000000000000000000064,1.0,0.000000,0.000000,0.333333,0.333333,0.192450
0x0000000000000000000000000000000000000800,2.0,0.000000,0.000000,0.000000,0.000000,0.000000
0x0000000000000000000000000000000000001010,7.0,0.142857,0.714286,0.523810,1.380952,0.290957
...,...,...,...,...,...,...
0xffffffff2ba8f66d4e51811c5190992176930278,133.0,18.463659,6.548872,3.701754,28.714286,7.831369
0xfffffffff15abf397da76f1dcc1a1604f45126db,67.0,28.676617,6.082090,2.860697,37.619403,14.067408
0xffffffffff2419497bf75e415bc9a7d446e05c0f,2.0,8.666667,27.500000,6.500000,42.666667,11.549812
0xffffffffff402b1b62421a978cf93a56453d1496,1.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [12]:
test_df.query('stdVal > 0')['totalVal'].quantile(.975)

25.333333333333332

In [69]:
decreasing_df = (
    test_df
    .query('stdVal > 0 and stdVal <= 10') #10 also 97.5th quantile
    .query('totalVal > 25') #97.5th quantile
    .sort_values(by = 'degree', ascending = False) #Tem que testar esses limiares aí que eu usei o 97.5º quantil 
)
decreasing_df

,degree,avgValue1,avgValue2,avgValue3,totalVal,stdVal
0x7a250d5630b4cf539739df2c5dacb4c659f2488d,323122.0,11.291720,11.785063,4.325878,27.402660,4.171446
0xa1d8d972560c2f8144af871db508f0b0b10a3fbf,12394.0,22.384057,23.095934,9.519068,54.999059,7.641400
0xd07dc4262bcdbf85190c01c996b4c06a461d2430,4425.0,12.803315,7.462147,4.836309,25.101770,4.059891
0xf6874c88757721a02f47592140905c4336dfbc61,3244.0,15.371147,19.653822,18.558364,53.583333,2.224848
0xed212a4a2e82d5ee0d62f70b5dee2f5ee0f10c5d,2880.0,6.451736,7.655556,11.032870,25.140162,2.374947
...,...,...,...,...,...,...
0xa431fba4027585b4e66fccadd70d0da1702d49c9,1.0,11.000000,12.000000,2.666667,25.666667,5.124379
0x39caf13a104ff567f71fd2a4c68c026fdb6e740b,1.0,1.000000,15.500000,10.666667,27.166667,7.383039
0x25194f5b023255db57dcaeb777fda8eddc3684ca,1.0,12.333333,9.000000,5.000000,26.333333,3.671714
0xa439083921877dea6e4001d9507455288c70b631,1.0,0.000000,16.000000,13.000000,29.000000,8.504901


In [74]:
test_df.query('stdVal > 0')['totalVal'].quantile(.025)

0.08333333333333333

In [75]:
increasing_df = (
    test_df
    .query('avgValue1 > 0 and avgValue2 > 0 and avgValue3 > 0')
    .query('stdVal > 0 and stdVal <= 10')
    .query('totalVal <= 0.08') #2.5th quantile
    .sort_values(by = 'degree', ascending = False)
)
increasing_df

,degree,avgValue1,avgValue2,avgValue3,totalVal,stdVal
0x042523db4f3effc33d2742022b2490258494f8b3,33928.0,0.014993,0.025805,0.008046,0.048844,0.008949
0x60b86af869f23aeb552fb7f3cabd11b829f6ab2f,31479.0,0.015640,0.026144,0.007518,0.049303,0.009338
0x000000c3cfd83e7f9d856bed82231e8a00a1b07f,14935.0,0.000446,0.030198,0.000603,0.031247,0.017132
0xf1ca9cb74685755965c7458528a36934df52a3ef,9932.0,0.002282,0.003574,0.003860,0.009716,0.000841
0x84a0856b038eaad1cc7e297cf34a7e72685a8693,5264.0,0.000127,0.000095,0.000887,0.001108,0.000448
...,...,...,...,...,...,...
0xb2c4f6db6d20059b2a2c29539b806b97d7c1e7dc,16.0,0.020833,0.031250,0.020833,0.072917,0.006014
0x32447853a6e6e39c902f973b9efac56af4434b16,16.0,0.020833,0.031250,0.020833,0.072917,0.006014
0xf9ff01cb2721510cb316d4e9cb3c44f98e22cc94,16.0,0.020833,0.031250,0.020833,0.072917,0.006014
0x6317464e3af082cc37f84f3083b0e73170fed5a5,16.0,0.020833,0.031250,0.020833,0.072917,0.006014


In [71]:
decreasing_df.query('degree > 3000') #Testar limiar de nó mínimo absoluto para relevância

,degree,avgValue1,avgValue2,avgValue3,totalVal,stdVal
0x7a250d5630b4cf539739df2c5dacb4c659f2488d,323122.0,11.291720,11.785063,4.325878,27.402660,4.171446
0xa1d8d972560c2f8144af871db508f0b0b10a3fbf,12394.0,22.384057,23.095934,9.519068,54.999059,7.641400
0xd07dc4262bcdbf85190c01c996b4c06a461d2430,4425.0,12.803315,7.462147,4.836309,25.101770,4.059891
0xf6874c88757721a02f47592140905c4336dfbc61,3244.0,15.371147,19.653822,18.558364,53.583333,2.224848


In [76]:
increasing_df.query('degree >= 3000') #Testar limiar de nó mínimo absoluto para relevância

,degree,avgValue1,avgValue2,avgValue3,totalVal,stdVal
0x042523db4f3effc33d2742022b2490258494f8b3,33928.0,0.014993,0.025805,0.008046,0.048844,0.008949
0x60b86af869f23aeb552fb7f3cabd11b829f6ab2f,31479.0,0.015640,0.026144,0.007518,0.049303,0.009338
0x000000c3cfd83e7f9d856bed82231e8a00a1b07f,14935.0,0.000446,0.030198,0.000603,0.031247,0.017132
0xf1ca9cb74685755965c7458528a36934df52a3ef,9932.0,0.002282,0.003574,0.003860,0.009716,0.000841
0x84a0856b038eaad1cc7e297cf34a7e72685a8693,5264.0,0.000127,0.000095,0.000887,0.001108,0.000448
0x14ff6ea1e1db3615a98f595443f831a8973311dd,3838.0,0.007817,0.007165,0.000782,0.015763,0.003887
0x758716e7e12f2f8b9e9d76d1e066d6f0ff0e299b,3802.0,0.037612,0.018937,0.005436,0.061985,0.016157
0x431b5a84acc1297eda88259f300262f1bc3a74f3,3686.0,0.002261,0.003391,0.047296,0.052948,0.025681
0x4a986a6dca6dbf99bc3d17f8d71afb0d60e740f8,3644.0,0.005488,0.014544,0.052964,0.072997,0.025206
